In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

def generate_insights_and_charts(excel_file_path: str, temp_dir: str = "temp"):
    """
    Reads the specified Excel file containing sheets:
      1) alerts
      2) ambulance
      3) firefighter
      4) police
      5) drone
      6) dispatch

    Generates a variety of metrics and charts, then saves PNGs to the 'temp' folder.
    Returns a dictionary of key statistics and derived metrics.
    """

    # Create temp folder if not exists
    os.makedirs(temp_dir, exist_ok=True)

    # ------------------------------
    # 1. Read Excel Sheets
    # ------------------------------
    alerts_df = pd.read_excel(excel_file_path, sheet_name="alerts")
    ambulance_df = pd.read_excel(excel_file_path, sheet_name="ambulance")
    firefighter_df = pd.read_excel(excel_file_path, sheet_name="firefighter")
    police_df = pd.read_excel(excel_file_path, sheet_name="police")
    drone_df = pd.read_excel(excel_file_path, sheet_name="drone")
    dispatch_df = pd.read_excel(excel_file_path, sheet_name="dispatch")

    # ------------------------------
    # 2. Basic Data Cleaning / Standardization
    # ------------------------------
    # Fix possible typos in alerts' "Responder Type" (e.g. "polie" -> "police")
    alerts_df["Responder Type"] = alerts_df["Responder Type"].replace({"polie": "police"})

    # Convert date/time columns to datetime where needed
    alerts_df["Timestamp"] = pd.to_datetime(alerts_df["Timestamp"], errors="coerce")
    dispatch_df["Dispatch time"] = pd.to_datetime(dispatch_df["Dispatch time"], errors="coerce")

    # Convert numeric columns from strings if needed (Response Time, Resolution Time)
    alerts_df["Response Time"] = pd.to_numeric(alerts_df["Response Time"], errors="coerce")
    alerts_df["Resolution Time"] = pd.to_numeric(alerts_df["Resolution Time"], errors="coerce")

    # Replace "NULL" with actual pandas NaN in dispatch sheet
    dispatch_df.replace("NULL", pd.NA, inplace=True)

    # ------------------------------
    # 3. Merge Additional Data from ambulance/police/firefighter/drone
    #    to dispatch to see which zone each responder is from.
    # ------------------------------

    # We'll suffix each zone column to clarify the service:
    # e.g., dispatch_df["Police Zone"], dispatch_df["Ambulance Zone"], etc.
    # First, we rename "Zone" in each reference df for clarity before merging
    ambulance_zones = ambulance_df.rename(columns={"Zone": "Ambulance Zone"})
    firefighter_zones = firefighter_df.rename(columns={"Zone": "Firefighter Zone"})
    police_zones = police_df.rename(columns={"Zone": "Police Zone"})
    drone_zones = drone_df.rename(columns={"Zone": "Drone Zone"})

    # Merge each zone reference into dispatch_df
    # We'll merge on e.g. dispatch_df["Police ID"] == police_zones["ID"]
    dispatch_df = dispatch_df.merge(
        police_zones[["ID", "Police Zone"]], 
        left_on="Police ID", right_on="ID", how="left", suffixes=("", "_p")
    ).drop(columns=["ID_p"])  # drop the duplicate ID column

    dispatch_df = dispatch_df.merge(
        ambulance_zones[["ID", "Ambulance Zone"]],
        left_on="Ambulance ID", right_on="ID", how="left", suffixes=("", "_a")
    ).drop(columns=["ID_a"])

    dispatch_df = dispatch_df.merge(
        firefighter_zones[["ID", "Firefighter Zone"]],
        left_on="Firefighter ID", right_on="ID", how="left", suffixes=("", "_f")
    ).drop(columns=["ID_f"])

    dispatch_df = dispatch_df.merge(
        drone_zones[["ID", "Drone Zone"]],
        left_on="Drone ID", right_on="ID", how="left", suffixes=("", "_d")
    ).drop(columns=["ID_d"])

    # Now dispatch_df has additional columns: "Police Zone", "Ambulance Zone", 
    # "Firefighter Zone", and "Drone Zone" for each dispatch row.

    # ------------------------------
    # 4. Derived Metrics & Merging with Alerts
    # ------------------------------
    # We'll also merge dispatch with alerts (so we can compare alert location and zone)
    merged_df = dispatch_df.merge(alerts_df, left_on="Alert ID", right_on="ID", how="left", suffixes=("", "_alert"))

    # Example: see if the zone "matches" the alert's location by partial match
    # We'll define a small helper that checks if zone text is in location text (case-insensitive).
    def zone_matches_location(zone: str, location: str) -> bool:
        if pd.isna(zone) or pd.isna(location):
            return False
        zone_clean = zone.lower().strip()
        loc_clean = location.lower().strip()
        return zone_clean in loc_clean  # basic substring check

    # We'll create columns: "Police_ZoneMatch", "Ambulance_ZoneMatch", etc.
    merged_df["Police_ZoneMatch"] = merged_df.apply(
        lambda row: zone_matches_location(row["Police Zone"], row["Location"]), axis=1
    )
    merged_df["Ambulance_ZoneMatch"] = merged_df.apply(
        lambda row: zone_matches_location(row["Ambulance Zone"], row["Location"]), axis=1
    )
    merged_df["Firefighter_ZoneMatch"] = merged_df.apply(
        lambda row: zone_matches_location(row["Firefighter Zone"], row["Location"]), axis=1
    )
    merged_df["Drone_ZoneMatch"] = merged_df.apply(
        lambda row: zone_matches_location(row["Drone Zone"], row["Location"]), axis=1
    )

    # We could see if matching zone correlates with lower response time
    # For that, we need the "Response Time" from the alerts (already in merged_df as "Response Time").

    # For demonstration, let's create a column "Any_ZoneMatch" if any of the responders match
    merged_df["Any_ZoneMatch"] = (
        merged_df["Police_ZoneMatch"] |
        merged_df["Ambulance_ZoneMatch"] |
        merged_df["Firefighter_ZoneMatch"] |
        merged_df["Drone_ZoneMatch"]
    )

    # Additional derived columns in alerts_df
    alerts_df["TimeRatio"] = alerts_df.apply(
        lambda row: (row["Resolution Time"] / row["Response Time"])
        if (row["Response Time"] and row["Response Time"] > 0) else pd.NA,
        axis=1
    )
    alerts_df["IsResolved"] = alerts_df["Status"].apply(lambda x: x == "resolved")
    severity_map = {"critical": 3, "high": 2, "warning": 1}
    alerts_df["SeverityCode"] = alerts_df["Severity"].map(severity_map)

    # ------------------------------
    # 5. Generate Key Stats
    # ------------------------------
    # A. Basic counts from alerts
    count_by_type = alerts_df["Type"].value_counts().to_dict()
    count_by_severity = alerts_df["Severity"].value_counts().to_dict()
    count_by_status = alerts_df["Status"].value_counts().to_dict()

    # B. Average response/resolution times
    avg_response_by_type = alerts_df.groupby("Type")["Response Time"].mean().to_dict()
    avg_response_by_severity = alerts_df.groupby("Severity")["Response Time"].mean().to_dict()
    avg_resolution_time_by_type = alerts_df.groupby("Type")["Resolution Time"].mean().to_dict()

    # C. Resolution stats
    resolution_rate_by_type = alerts_df.groupby("Type")["IsResolved"].mean().to_dict()

    # D. Dispatch usage stats: how many times each zone was used for each service
    # We can count the non-null references to each zone
    # or get the distribution of Police Zone, Ambulance Zone, etc. from merged_df
    zone_counts_police = merged_df["Police Zone"].value_counts(dropna=True).to_dict()
    zone_counts_ambulance = merged_df["Ambulance Zone"].value_counts(dropna=True).to_dict()
    zone_counts_firefighter = merged_df["Firefighter Zone"].value_counts(dropna=True).to_dict()
    zone_counts_drone = merged_df["Drone Zone"].value_counts(dropna=True).to_dict()

    # E. Zone match effect on response time (optional quick check)
    match_df = merged_df.dropna(subset=["Response Time"])  # only rows with a valid response time
    avg_resp_time_match = match_df.groupby("Any_ZoneMatch")["Response Time"].mean().to_dict()

    key_metrics = {
        # Alerts Stats
        "count_by_type": count_by_type,
        "count_by_severity": count_by_severity,
        "count_by_status": count_by_status,
        "avg_response_by_type": avg_response_by_type,
        "avg_response_by_severity": avg_response_by_severity,
        "avg_resolution_time_by_type": avg_resolution_time_by_type,
        "resolution_rate_by_type": resolution_rate_by_type,
        # Dispatch Stats
        "zone_counts_police": zone_counts_police,
        "zone_counts_ambulance": zone_counts_ambulance,
        "zone_counts_firefighter": zone_counts_firefighter,
        "zone_counts_drone": zone_counts_drone,
        # Zone Matching
        "avg_response_time_zone_match": avg_resp_time_match,
    }

    # ------------------------------
    # 6. Visualizations
    # ------------------------------
    sns.set_style("whitegrid")

    # (Same as the original code, plus some new ones for zone usage.)

    # A. Alerts by Type
    plt.figure(figsize=(6,4))
    sns.countplot(data=alerts_df, x="Type", palette="Set2")
    plt.title("Count of Alerts by Type")
    plt.xlabel("Alert Type")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(os.path.join(temp_dir, "alerts_by_type.png"))
    plt.close()

    # B. Alerts by Severity
    plt.figure(figsize=(6,4))
    sns.countplot(data=alerts_df, x="Severity", palette="Set3")
    plt.title("Count of Alerts by Severity")
    plt.xlabel("Severity")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(os.path.join(temp_dir, "alerts_by_severity.png"))
    plt.close()

    # C. Alerts by Status
    plt.figure(figsize=(6,4))
    sns.countplot(data=alerts_df, x="Status", palette="Set1")
    plt.title("Count of Alerts by Status")
    plt.xlabel("Status")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(os.path.join(temp_dir, "alerts_by_status.png"))
    plt.close()

    # D. Average Response Time by Type
    avg_response = alerts_df.groupby("Type")["Response Time"].mean().reset_index()
    plt.figure(figsize=(6,4))
    sns.barplot(data=avg_response, x="Type", y="Response Time", palette="coolwarm")
    plt.title("Average Response Time by Alert Type")
    plt.xlabel("Alert Type")
    plt.ylabel("Avg. Response Time (min)")
    plt.tight_layout()
    plt.savefig(os.path.join(temp_dir, "avg_response_by_type.png"))
    plt.close()

    # E. Average Resolution Time by Type
    avg_resolution = alerts_df.groupby("Type")["Resolution Time"].mean().reset_index()
    plt.figure(figsize=(6,4))
    sns.barplot(data=avg_resolution, x="Type", y="Resolution Time", palette="flare")
    plt.title("Average Resolution Time by Alert Type")
    plt.xlabel("Alert Type")
    plt.ylabel("Avg. Resolution Time (min)")
    plt.tight_layout()
    plt.savefig(os.path.join(temp_dir, "avg_resolution_by_type.png"))
    plt.close()

    # F. Distribution of Response Times
    plt.figure(figsize=(6,4))
    sns.histplot(data=alerts_df, x="Response Time", kde=True)
    plt.title("Distribution of Response Times")
    plt.xlabel("Response Time (min)")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(os.path.join(temp_dir, "dist_response_time.png"))
    plt.close()

    # G. Dispatch: Police Zones usage
    zone_counts_p_df = pd.DataFrame(list(zone_counts_police.items()), columns=["Zone", "Count"])
    plt.figure(figsize=(6,4))
    sns.barplot(data=zone_counts_p_df, x="Zone", y="Count", palette="viridis")
    plt.title("Police Zone Usage")
    plt.xlabel("Zone")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(temp_dir, "police_zone_usage.png"))
    plt.close()

    # (Similarly for ambulance, firefighter, and drone if you want individual charts)

    # H. Zone Match vs. Response Time
    # We'll do a simple boxplot to see if "Any_ZoneMatch" has lower or higher response time
    match_df = merged_df.dropna(subset=["Response Time"]).copy()
    match_df["Any_ZoneMatch"] = match_df["Any_ZoneMatch"].astype(bool)
    plt.figure(figsize=(6,4))
    sns.boxplot(data=match_df, x="Any_ZoneMatch", y="Response Time", palette="Set2")
    plt.title("Response Time vs. Zone Match (Any Responder)")
    plt.xlabel("Any Responder Zone Matched Alert Location?")
    plt.ylabel("Response Time (min)")
    plt.tight_layout()
    plt.savefig(os.path.join(temp_dir, "zone_match_response_time.png"))
    plt.close()

    # ------------------------------
    # Return / Print Key Metrics
    # ------------------------------
    return key_metrics


In [2]:
excel_path = "MLX_Reports-2.xlsx"  # Adjust path
metrics = generate_insights_and_charts(excel_path)
print("Key metrics and derived insights:\n", metrics)

C:\Users\Suhail\AppData\Local\Temp\ipykernel_42548\2157126758.py:195: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=alerts_df, x="Type", palette="Set2")
C:\Users\Suhail\AppData\Local\Temp\ipykernel_42548\2157126758.py:205: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=alerts_df, x="Severity", palette="Set3")
C:\Users\Suhail\AppData\Local\Temp\ipykernel_42548\2157126758.py:215: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=alerts_df, x="Status", palette="Set1")
C:\Users\Suhail\AppData\Local\Temp\ipykernel_42548\2157126758

Key metrics and derived insights:
 {'count_by_type': {'fire': 4, 'assault': 3}, 'count_by_severity': {'critical': 4, 'warning': 2, 'high': 1}, 'count_by_status': {'resolved': 3, 'in progress': 3, 'unresolved': 1}, 'avg_response_by_type': {'assault': 9.666666666666666, 'fire': 24.75}, 'avg_response_by_severity': {'critical': 9.75, 'high': 17.0, 'warning': 36.0}, 'avg_resolution_time_by_type': {'assault': 23.0, 'fire': 10.5}, 'resolution_rate_by_type': {'assault': 0.6666666666666666, 'fire': 0.25}, 'zone_counts_police': {'Hamdan Street': 1, 'Khalidiya Mall': 1, 'Corniche Road': 1, 'Reem Island': 1, 'Yas Mall': 1, 'Al Mushrif': 1}, 'zone_counts_ambulance': {'Hamdan Street': 1, 'Khalidiya Mall': 1, 'Al Mushrif': 1}, 'zone_counts_firefighter': {'Hamdan Street': 1, 'Khalidiya Mall': 1, 'Corniche Road': 1, 'Yas Mall': 1, 'Al Raha Beach': 1}, 'zone_counts_drone': {'Hamdan Street': 1, 'Khalidiya Mall': 1, 'Corniche Road': 1, 'Yas Mall': 1, 'Al Raha Beach': 1}, 'avg_response_time_zone_match': {F

C:\Users\Suhail\AppData\Local\Temp\ipykernel_42548\2157126758.py:274: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=match_df, x="Any_ZoneMatch", y="Response Time", palette="Set2")
